In [18]:
import requests as rq
import pandas as pd
import datetime as dt
import os

## Descarga de datos

Variables

In [386]:
url = 'https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_aperturas.xls'
today_date = dt.datetime.today().strftime('%Y-%m-%d') 


In [387]:
# Descarga de datos mediante el requests
try:
    r = rq.get(url)
    r.raise_for_status()
except rq.exceptions.HTTPError as err:
    print(err)



# Guardado de datos en el directorio /Data y se renombra el archivo agregando la fecha de descarga
if len(os.listdir('data')) == 0:
    open(f'Data/sh_ipc_aperturas_{today_date}.xls', 'wb').write(r.content)
    print('El archivo se ha descargado correctamente')
else:
    for file in os.scandir('Data'):
        if file.name == f'sh_ipc_aperturas_{today_date}.xls':
            print('El archivo ya existe')
            break
        else:
            open(f'Data/sh_ipc_aperturas_{today_date}.xls', 'wb').write(r.content)
            print('El archivo se ha descargado correctamente')      
   



El archivo se ha descargado correctamente


## Tratamiento de datos

Variables

In [388]:
file = f'Data/sh_ipc_aperturas_{today_date}.xls'

Creacion de datasets para su posterior analisis

In [389]:
df = pd.read_excel(file, sheet_name='Índices aperturas', skiprows=4, parse_dates=True)


In [390]:
df.shape

(297, 72)

In [391]:
df.info() #Tipo de datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 72 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   269 non-null    object
 1   Unnamed: 1   266 non-null    object
 2   Unnamed: 2   266 non-null    object
 3   Unnamed: 3   266 non-null    object
 4   Unnamed: 4   266 non-null    object
 5   Unnamed: 5   266 non-null    object
 6   Unnamed: 6   266 non-null    object
 7   Unnamed: 7   266 non-null    object
 8   Unnamed: 8   266 non-null    object
 9   Unnamed: 9   266 non-null    object
 10  Unnamed: 10  266 non-null    object
 11  Unnamed: 11  266 non-null    object
 12  Unnamed: 12  266 non-null    object
 13  Unnamed: 13  266 non-null    object
 14  Unnamed: 14  266 non-null    object
 15  Unnamed: 15  266 non-null    object
 16  Unnamed: 16  266 non-null    object
 17  Unnamed: 17  266 non-null    object
 18  Unnamed: 18  266 non-null    object
 19  Unnamed: 19  266 non-null    

In [392]:
df.isna().sum().sum() #Cantidad de valores nulos

2229

- La pestaña de datos 'indices aperturas' contiene varias row datos NaN, por lo que se eliminan para facilitar el analisis.
- Las columnas de datos estan en string, por lo que se convertiran a float para poder realizar operaciones matematicas.
- Se nombran las columnas de la pestaña 'indices aperturas' para facilitar el analisis.
- Debido a la complejidad del dataset y su estructura se decide dividir las regiones en dataset individuales.
- Realizamos un transpose de los datasets para facilitar el analisis. Quedando como nombre de los campos el nombre del indice y como indice la fecha.


In [393]:
df = df.dropna()
df.columns = ['Columna_'+str(i) for i in range(1, len(df.columns)+1)]
df = df.reset_index(drop=True)
df.head()

,Columna_1,Columna_2,Columna_3,Columna_4,Columna_5,Columna_6,Columna_7,Columna_8,Columna_9,Columna_10,...,Columna_63,Columna_64,Columna_65,Columna_66,Columna_67,Columna_68,Columna_69,Columna_70,Columna_71,Columna_72
0,Región GBA,2016-12-01 00:00:00,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,2017-04-01 00:00:00,2017-05-01 00:00:00,2017-06-01 00:00:00,2017-07-01 00:00:00,2017-08-01 00:00:00,...,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,2022-05-01 00:00:00,2022-06-01 00:00:00,2022-07-01 00:00:00,2022-08-01 00:00:00,2022-09-01 00:00:00,2022-10-01 00:00:00
1,Nivel general,100,101.313,103.8085,106.2627,109.0613,110.4607,111.9943,113.9199,115.6031,...,593.4219,620.5007,661.8688,702.829,736.3637,776.6701,833.8894,891.851,945.6836,1007.6275
2,Alimentos y bebidas no alcohólicas,100,101.3069,103.1646,106.7878,109.2144,110.3615,111.7923,113.1579,115.6549,...,627.7655,681.4654,722.8456,765.7118,799.1293,838.7821,888.2122,949.1534,1016.1594,1084.2719
3,Alimentos,100,101.2848,103.3191,107.1358,109.6145,110.6764,112.1594,113.3581,115.9763,...,645.0681,703.6564,747.2396,792.709,826.4546,866.4017,917.1386,980.6468,1051.1431,1119.2361
4,Pan y cereales,100,101.5263,103.7498,106.6043,108.8026,109.7644,111.8159,113.7735,115.1724,...,576.2742,609.602,680.0321,736.4364,767.4221,805.117,861.7042,921.9103,974.7834,1034.9946


Obtenemos las regiones y sus index

In [394]:
regiones = {}
for row in df.Columna_1:
    if 'Región' in row:
        regiones[row] = df[df.Columna_1 == row].index[0]

print(regiones)
        


{'Región GBA': 0, 'Región Pampeana': 46, 'Región Noroeste': 90, 'Región Noreste': 134, 'Región Cuyo': 178, 'Región Patagonia': 222}


### Generamos los dataset por region

Cada dataset contiene los datos de una region, con sus respectivos indices de apertura. Se guardan en formato csv en /Dataframe.

In [413]:

region_gba = df[regiones['Región GBA'] : regiones['Región Pampeana']]


region_gba = region_gba.transpose()
region_gba.columns = region_gba.iloc[0]
region_gba = region_gba.drop(region_gba.index[0])
region_gba = region_gba.reset_index(drop=True)
region_gba.rename(columns={'Región GBA': 'Fecha'}, inplace=True)
region_gba['Fecha'] = pd.to_datetime(region_gba['Fecha'], format='%Y-%m-%d').apply(lambda x: x.strftime('%Y-%m'))

region_gba.to_csv(f'Dataframe/region_gba_{today_date}.csv', index=False)
region_gba.columns

Index(['Fecha', 'Nivel general', 'Alimentos y bebidas no alcohólicas',
       'Alimentos', 'Pan y cereales', 'Carnes y derivados',
       'Leche, productos lácteos y huevos', 'Aceites, grasas y manteca',
       'Frutas', 'Verduras, tubérculos y legumbres',
       'Azúcar, dulces, chocolate, golosinas, etc.', 'Bebidas no alcohólicas',
       'Café, té, yerba y cacao', 'Aguas minerales, bebidas gaseosas y jugos',
       'Bebidas alcohólicas y tabaco', 'Bebidas alcohólicas', 'Tabaco',
       'Prendas de vestir y calzado', 'Prendas de vestir y materiales',
       'Calzado', 'Vivienda, agua, electricidad, gas y otros combustibles',
       'Alquiler de la vivienda y gastos conexos', 'Alquiler de la vivienda',
       'Mantenimiento y reparación de la vivienda',
       'Electricidad, gas y otros combustibles',
       'Equipamiento y mantenimiento del hogar',
       'Bienes y servicios para la conservación del hogar', 'Salud',
       'Productos medicinales, artefactos y equipos para la salud',


In [414]:
region_pampeana = df[regiones['Región Pampeana'] : regiones['Región Noroeste']]
region_pampeana = region_pampeana.transpose()
region_pampeana.columns = region_pampeana.iloc[0]
region_pampeana = region_pampeana.drop(region_pampeana.index[0])
region_pampeana = region_pampeana.reset_index(drop=True)
region_pampeana.rename(columns={'Región Pampeana': 'Fecha'}, inplace=True)
region_pampeana['Fecha'] = pd.to_datetime(region_pampeana['Fecha'], format='%Y-%m-%d').apply(lambda x: x.strftime('%Y-%m'))

region_pampeana.to_csv(f'Dataframe/region_pampeana_{today_date}.csv', index=False)
region_pampeana.columns

Index(['Fecha', 'Nivel general', 'Alimentos y bebidas no alcohólicas',
       'Alimentos', 'Pan y cereales', 'Carnes y derivados',
       'Leche, productos lácteos y huevos', 'Aceites, grasas y manteca',
       'Frutas', 'Verduras, tubérculos y legumbres',
       'Azúcar, dulces, chocolate, golosinas, etc.', 'Bebidas no alcohólicas',
       'Café, té, yerba y cacao', 'Aguas minerales, bebidas gaseosas y jugos',
       'Bebidas alcohólicas y tabaco', 'Bebidas alcohólicas', 'Tabaco',
       'Prendas de vestir y calzado', 'Prendas de vestir y materiales',
       'Calzado', 'Vivienda, agua, electricidad, gas y otros combustibles',
       'Alquiler de la vivienda y gastos conexos', 'Alquiler de la vivienda',
       'Electricidad, gas y otros combustibles',
       'Equipamiento y mantenimiento del hogar',
       'Bienes y servicios para la conservación del hogar', 'Salud',
       'Productos medicinales, artefactos y equipos para la salud',
       'Gastos de prepagas', 'Transporte', 'Adquisic

In [409]:
region_noroeste = df[regiones['Región Noroeste'] : regiones['Región Noreste']]
region_noroeste = region_noroeste.transpose()
region_noroeste.columns = region_noroeste.iloc[0]
region_noroeste = region_noroeste.drop(region_noroeste.index[0])
region_noroeste = region_noroeste.reset_index(drop=True)
region_noroeste.rename(columns={'Región Noroeste': 'Fecha'}, inplace=True)
region_noroeste['Fecha'] = pd.to_datetime(region_noroeste['Fecha'], format='%Y-%m-%d').apply(lambda x: x.strftime('%Y-%m'))

region_noroeste.to_csv(f'Dataframe/region_noroeste_{today_date}.csv', index=False)
region_noroeste.columns

Index(['Fecha', 'Nivel general', 'Alimentos y bebidas no alcohólicas',
       'Alimentos', 'Pan y cereales', 'Carnes y derivados',
       'Leche, productos lácteos y huevos', 'Aceites, grasas y manteca',
       'Frutas', 'Verduras, tubérculos y legumbres',
       'Azúcar, dulces, chocolate, golosinas, etc.', 'Bebidas no alcohólicas',
       'Café, té, yerba y cacao', 'Aguas minerales, bebidas gaseosas y jugos',
       'Bebidas alcohólicas y tabaco', 'Bebidas alcohólicas', 'Tabaco',
       'Prendas de vestir y calzado', 'Prendas de vestir y materiales',
       'Calzado', 'Vivienda, agua, electricidad, gas y otros combustibles',
       'Alquiler de la vivienda y gastos conexos', 'Alquiler de la vivienda',
       'Electricidad, gas y otros combustibles',
       'Equipamiento y mantenimiento del hogar',
       'Bienes y servicios para la conservación del hogar', 'Salud',
       'Productos medicinales, artefactos y equipos para la salud',
       'Gastos de prepagas', 'Transporte', 'Adquisic

In [411]:
region_noreste = df[regiones['Región Noreste'] : regiones['Región Cuyo']]
region_noreste = region_noreste.transpose()
region_noreste.columns = region_noreste.iloc[0]
region_noreste = region_noreste.drop(region_noreste.index[0])
region_noreste = region_noreste.reset_index(drop=True)
region_noreste.rename(columns={'Región Noreste': 'Fecha'}, inplace=True)
region_noreste['Fecha'] = pd.to_datetime(region_noreste['Fecha'], format='%Y-%m-%d').apply(lambda x: x.strftime('%Y-%m'))

region_noreste.to_csv(f'Dataframe/region_noreste_{today_date}.csv', index=False)
region_noreste.head()

Columna_1,Fecha,Nivel general,Alimentos y bebidas no alcohólicas,Alimentos,Pan y cereales,Carnes y derivados,"Leche, productos lácteos y huevos","Aceites, grasas y manteca",Frutas,"Verduras, tubérculos y legumbres",...,Comunicación,Servicios de telefonía e internet,Recreación y cultura,Servicios recreativos y culturales,"Periódicos, diarios, revistas, libros y artículos de papelería",Educación,Restaurantes y hoteles,Restaurantes y comidas fuera del hogar,Bienes y servicios varios,Cuidado personal
0,2016-12,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1,2017-01,101.6727,101.7326,101.7363,101.7136,100.2153,102.9346,101.8631,103.6335,104.9004,...,102.7252,102.9493,103.633,105.1661,102.0618,101.6601,104.2939,103.8556,102.0562,101.9754
2,2017-02,103.4617,103.3712,103.3208,102.894,100.6351,105.4341,105.2686,104.5632,109.6906,...,107.0976,107.4178,105.6846,107.4875,107.4075,109.7825,108.3164,107.9867,103.2166,103.1591
3,2017-03,105.988,105.0298,105.0519,104.6449,102.9874,108.627,104.1701,96.1416,111.7862,...,110.7662,110.9125,105.7406,108.8534,107.0263,123.958,110.4746,110.5993,104.5375,104.5162
4,2017-04,108.3473,107.9352,108.043,106.3181,106.2397,113.5727,106.743,97.9005,115.1872,...,120.0581,120.1146,109.1455,109.9417,113.3606,125.2085,110.876,111.0573,107.7262,107.5726


In [404]:
region_cuyo = df[regiones['Región Cuyo'] : regiones['Región Patagonia']]
region_cuyo = region_cuyo.transpose()
region_cuyo.columns = region_cuyo.iloc[0]
region_cuyo = region_cuyo.drop(region_cuyo.index[0])
region_cuyo = region_cuyo.reset_index(drop=True)
region_cuyo.rename(columns={'Región Cuyo': 'Fecha'}, inplace=True)
region_cuyo['Fecha'] = pd.to_datetime(region_cuyo['Fecha'], format='%Y-%m-%d').apply(lambda x: x.strftime('%Y-%m'))

region_cuyo.to_csv(f'Dataframe/region_cuyo_{today_date}.csv', index=False)
region_cuyo.head()

Columna_1,Fecha,Nivel general,Alimentos y bebidas no alcohólicas,Alimentos,Pan y cereales,Carnes y derivados,"Leche, productos lácteos y huevos","Aceites, grasas y manteca",Frutas,"Verduras, tubérculos y legumbres",...,Comunicación,Servicios de telefonía e internet,Recreación y cultura,Servicios recreativos y culturales,"Periódicos, diarios, revistas, libros y artículos de papelería",Educación,Restaurantes y hoteles,Restaurantes y comidas fuera del hogar,Bienes y servicios varios,Cuidado personal
0,2016-12,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1,2017-01,101.7074,101.4532,101.1509,101.6816,100.1244,102.0523,104.4466,102.9104,100.4237,...,102.7973,102.785,101.494,102.4171,102.17,100,101.8587,100.8187,102.0274,101.8622
2,2017-02,103.2652,103.2726,102.8614,103.4011,100.1186,104.8258,112.0237,103.5016,105.6971,...,105.0927,105.0145,103.0695,106.161,104.8766,102.6468,103.7947,102.8799,103.145,102.7663
3,2017-03,105.9238,105.6107,105.3056,106.075,102.5817,107.9784,111.199,101.049,109.7505,...,108.0971,107.8804,105.7508,111.317,107.673,112.7186,104.0537,103.999,105.5914,104.608
4,2017-04,109.4506,108.662,108.5645,110.4638,106.0002,112.5432,110.995,99.4502,114.0377,...,117.3494,117.4702,108.008,112.3921,109.5933,123.0399,106.7437,106.9207,106.9862,105.8914


In [405]:
region_patagonica = df[regiones['Región Patagonia'] : ]
region_patagonica = region_patagonica.transpose()
region_patagonica.columns = region_patagonica.iloc[0]
region_patagonica = region_patagonica.drop(region_patagonica.index[0])
region_patagonica = region_patagonica.reset_index(drop=True)
region_patagonica.rename(columns={'Región Patagonia': 'Fecha'}, inplace=True)
region_patagonica['Fecha'] = pd.to_datetime(region_patagonica['Fecha'], format='%Y-%m-%d').apply(lambda x: x.strftime('%Y-%m'))

region_patagonica.to_csv(f'Dataframe/region_patagonica_{today_date}.csv', index=False)
region_patagonica.tail()

Columna_1,Fecha,Nivel general,Alimentos y bebidas no alcohólicas,Alimentos,Pan y cereales,Carnes y derivados,"Leche, productos lácteos y huevos","Aceites, grasas y manteca",Frutas,"Verduras, tubérculos y legumbres",...,Comunicación,Servicios de telefonía e internet,Recreación y cultura,Servicios recreativos y culturales,"Periódicos, diarios, revistas, libros y artículos de papelería",Educación,Restaurantes y hoteles,Restaurantes y comidas fuera del hogar,Bienes y servicios varios,Cuidado personal
66,2022-06,782.142,803.4338,808.3129,794.4008,842.5337,831.3289,839.5017,745.4129,733.0548,...,558.6684,535.5844,695.5721,587.5368,724.3439,568.2106,736.5286,736.2363,664.9833,685.7009
67,2022-07,845.0397,862.1406,865.8464,851.0614,897.51,890.8966,900.8365,799.835,799.7184,...,591.5428,566.9438,762.9884,621.719,793.7224,595.2195,812.549,801.2937,713.5734,737.9148
68,2022-08,899.4604,920.7258,924.9011,901.9685,940.3121,966.6064,954.4469,875.4432,891.0488,...,624.4615,596.0992,819.2832,649.5198,899.025,624.5188,865.6561,856.794,773.74,804.3695
69,2022-09,954.6011,982.087,987.464,949.3995,986.1092,1032.9942,1036.8103,944.4737,1030.8057,...,645.622,617.3648,872.3573,698.7717,971.8328,638.2452,929.1266,925.8952,819.7012,853.0501
70,2022-10,1017.5863,1053.1016,1058.865,1016.3909,1032.9051,1111.3273,1131.0797,1046.6721,1173.3552,...,713.4248,683.2761,931.6448,731.598,1051.661,661.0015,1000.7229,994.1824,905.4486,941.0044
